In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, WebSearchTool, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel, Field
import asyncio
from IPython.display import display, Markdown
from typing import Dict
from sendgrid.helpers.mail import Mail, Email, To, Content
import sendgrid


In [ ]:
load_dotenv(override=True)

In [ ]:
ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1/"

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

anthropic_client = AsyncOpenAI(base_url=ANTHROPIC_BASE_URL, api_key=anthropic_api_key)

anthropic_model = OpenAIChatCompletionsModel(model="claude-3-7-sonnet-latest", openai_client=anthropic_client)


In [ ]:
INSTRUCTIONS = "You are a researcher. You are doing research about quantum computing." \
"Given the search term, you search the web for the answer. Produce a concise and clear epplanation of the results." \
"Capture the main points and explain them in simple terms." \
"Add links to the sources you used." \
"Provide no more than 3 sources.";

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini"
)

In [ ]:
NUMBER_OF_SEARCHES = 3

INSTRUCTIONS = f"You are a research assistant. Given a query, come up with a set of web searches \
to perform the best answer to the query. You can perform up to {NUMBER_OF_SEARCHES} searches. ";

print(INSTRUCTIONS)

class WebSearchItem(BaseModel):
    reason: str =  Field(description="The reason for why this search was performed.")
    query: str = Field(description="The search query used.")


class WebSearchOutput(BaseModel):
    search_items: list[WebSearchItem] = Field(description="List of search items with reasons and queries.")

planner_agent = Agent(
    name="Search planner agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchOutput
)



In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "Using the provided summary of research findings, compose a detailed report that elaborates on the key points. "
    "Ensure the report is well-structured, clear, and provides in-depth insights into the research topic."
)
class ReportData(BaseModel):
    summary: str = Field(description="A concise summary of the research findings.")
    report: str = Field(description="A detailed report of the research findings.")

report_agent = Agent(
    name="Report agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body"""

    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("mgredziak@gmail.com")  # Change to your verified sender
    to_email = To("michal.gredziak@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    subject = subject

    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status","success"}


In [ ]:
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_converter = Agent(name="HTML converter",
    instructions=html_instructions,
    model="gpt-4o-mini")

html_tool = html_converter.as_tool(
    name="html_converter",
    description="Convert a text email body to an HTML email body."
)

In [ ]:
tools = [send_html_email, html_tool]

In [ ]:
INSTRUCTIONS = "You are an email formatter and sender. Given the email content, convert it to HTML format and send it to the specified recipient.\
    You first use html converter tool to convert the email content to html. Then you use sendgrid to send the email."

emailAgent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    tools=tools,
    handoff_description="Convert an email to html and send it"
)


In [ ]:
async def plan_search(query: str):
    """Use the planner agent to create a search plan."""
    print("Planning search...")
    result = await Runner.run(planner_agent, f"Query:  {query}")
    print(f"Will perform {len(result.final_output.search_items)} searches.")
    return result.final_output

In [ ]:
async def execute_search_plan(search_plan: WebSearchOutput):
    """Execute the search plan using the search agent."""
    print("Executing search plan...")
    all_results = []
    for item in search_plan.search_items:
        print(f"Performing search for: {item.query} (Reason: {item.reason})")
        result = await Runner.run(search_agent, item.query)
        all_results.append(result.final_output)
    return all_results

In [ ]:
async def summarize_search_results(search_result: str):
    """Summarize the search results using the search agent."""
    print("Summarizing search results...")
    summary_instructions = "Summarize the following search results into a concise explanation with main points and sources."
    result = await Runner.run(report_agent, f"{summary_instructions}\n\n{search_result}")
    return result.final_output

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    search_plan = await plan_search(query)
    search_results = await execute_search_plan(search_plan)
    print(f"Search results: {search_results}")
    summary = await summarize_search_results(search_results)


In [ ]:
display(Markdown(summary.report))